Пятое место соревнования -> https://ods.ai/competitions/lacmus-cvc-soc2021

Решение основанно на YOLO v5 -> https://github.com/ultralytics/yolov5
Оснвоная работа была проделанна над поиском гиперпараметров и выборе размера нейросети 

Что не сработало:

-не сработали более тяжёлые версии yolov5m, yolov5l

-не сработало обучение на предворительно кропнутых изображениях



In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os
import shutil
import xml.etree.ElementTree as ET
import cv2

Загрузка датасета (data_lacmus.zip). Распаковка, конвертирвоание разметки в формат YOLO. Загрузка лучших весов(lacmus_settins.zip).


In [2]:
# !rm -r ./github.com/
# !rm -r ./lacmus_settings/
# !rm -r ./data/
# !rm -r ./predict/

In [3]:
%%time
#Примерное время выполнение -> 8 минут
gdd.download_file_from_google_drive(file_id='1V94GWXpTh-OgajEA40vUBNEiASvKYVG0',
                                    dest_path='./data/data_lacmus.zip',
                                    unzip=True)
!rm ./data/data_lacmus.zip
gdd.download_file_from_google_drive(file_id='1CoOySjzzLLmSwY2z6nkaY0pA2VFDG7bb',
                                    dest_path='./lacmus_settngs.zip',
                                    unzip=True)
!rm ./lacmus_settngs.zip


Unzipping...Done.
Unzipping...Done.
CPU times: user 1min 12s, sys: 45.5 s, total: 1min 58s
Wall time: 7min 44s


In [4]:
def get_imgSize_and_list_of_yxyx(xml_path):
    mytree = ET.parse(xml_path)
    people=[e for e in  list(mytree.iter()) if e.tag=="object" ]
    xyxy=[]
    for p in people:
        y1=int(p[4][0].text)
        x1=int(p[4][1].text)
        y2=int(p[4][2].text)
        x2=int(p[4][3].text)
        xyxy.append(((x1,y1),(x2,y2)))
    size_object=mytree.find("size")
    size=(int(size_object[1].text),int(size_object[0].text))
    return size, xyxy
def get_xywh_from_point(size,points):
    w,h=size
    x=((points[0][0]+points[1][0])/2)/w
    y=((points[0][1]+points[1][1])/2)/h
    im_w=abs((points[0][0]-points[1][0]))/w
    im_h=abs((points[0][1]-points[1][1]))/h
    return x,y,im_w,im_h

In [6]:
%%time
anotation_path='./data/TrainingData/Annotations'
labels_dir='./data/TrainingData/labels'
os.makedirs(labels_dir,exist_ok=True)
for xml_name in os.listdir(anotation_path):
  id,_ = os.path.splitext(xml_name)
  img_size, points_yxyx=get_imgSize_and_list_of_yxyx(os.path.join(anotation_path,xml_name))
  xywhs=[get_xywh_from_point(img_size,xyxy_single) for xyxy_single in points_yxyx]
  f=open(os.path.join(labels_dir,id+'.txt'),'w')
  for box in xywhs:
    f.write(" ".join(['0',*map(str,box)])+'\n')
  f.close()

CPU times: user 219 ms, sys: 115 ms, total: 334 ms
Wall time: 339 ms


In [7]:
%%time
#Примерное время выполнение -> 3 минуты
dataset_images_path='./data/images'
dataset_labels_path='./data/labels'
os.makedirs(dataset_images_path,exist_ok=True)
os.makedirs(dataset_labels_path, exist_ok=True)
train_file=open('./data/TrainingData/ImageSets/Main/train.txt','r')
for id in train_file.readlines():
  id=id.strip()
  images_train_path=os.path.join(dataset_images_path,'train')
  labels_train_path=os.path.join(dataset_labels_path,'train')
  os.makedirs(images_train_path,exist_ok=True)  
  os.makedirs(labels_train_path,exist_ok=True)
  shutil.copy(os.path.join('./data/TrainingData/JPEGImages',id+'.jpg')
              ,os.path.join(images_train_path,id+'.jpg'))
  shutil.copy(os.path.join('./data/TrainingData/labels/',id+'.txt')
              ,os.path.join(labels_train_path,id+'.txt'))    
  
val_file=open('./data/TrainingData/ImageSets/Main/val.txt','r')
for id in val_file.readlines():
  id=id.strip()
  images_train_path=os.path.join(dataset_images_path,'valid')
  labels_train_path=os.path.join(dataset_labels_path,'valid')
  os.makedirs(images_train_path,exist_ok=True)  
  os.makedirs(labels_train_path,exist_ok=True)
  shutil.move(os.path.join('./data/TrainingData/JPEGImages',id+'.jpg')
              ,os.path.join(images_train_path,id+'.jpg'))
  shutil.move(os.path.join('./data/TrainingData/labels/',id+'.txt')
              ,os.path.join(labels_train_path,id+'.txt'))    
  

CPU times: user 2.82 s, sys: 11.8 s, total: 14.7 s
Wall time: 2min 37s


Клонирование YOLO5 и обучение с выбраными гиперпараметрами

In [8]:
%%time
!git clone https://github.com/ultralytics/yolov5 

Cloning into 'yolov5'...
remote: Enumerating objects: 9237, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 9237 (delta 13), reused 13 (delta 2), pack-reused 9205
Receiving objects: 100% (9237/9237), 9.73 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (6401/6401), done.
CPU times: user 38 ms, sys: 8.92 ms, total: 46.9 ms
Wall time: 2.23 s


In [10]:
!python yolov5/train.py --hyp ./lacmus_settings/hyp_lacmus.yaml --batch-size 2 --worker 2 --data ./lacmus_settings/dataset_lacmus.yaml --weights yolov5s.pt --imgsz 1984 --epochs 10


train: weights=yolov5s.pt, cfg=, data=./lacmus_settings/dataset_lacmus.yaml, hyp=./lacmus_settings/hyp_lacmus.yaml, epochs=10, batch_size=2, imgsz=1984, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=2, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-414-g0d8a184 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h

Предсказание с использование лучших весов.

In [11]:
!python yolov5/detect.py --augment --weights ./lacmus_settings/best_lacmus.pt --source ./data/EmptyPart/ --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf

detect: weights=['./lacmus_settings/best_lacmus.pt'], source=./data/EmptyPart/, imgsz=[1984, 1984], conf_thres=0.05, iou_thres=0.01, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=predict, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-414-g0d8a184 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients
image 1/235 /content/data/EmptyPart/0.jpg: 1504x1984 3 persons, Done. (0.348s)
image 2/235 /content/data/EmptyPart/1.jpg: 1504x1984 4 persons, Done. (0.306s)
image 3/235 /content/data/EmptyPart/10.jpg: 1120x1984 2 persons, Done. (0.225s)
image 4/235 /content/data/EmptyPart/100.jpg: 1504x1984 1 person, Done. (0.287s)
image 5/235 /content/data/EmptyPart/101.j

Создание финального сабмита best_submit.csv

In [12]:
def convert_yolo_to_pixels(size,yolo_string):
    s=[float(s) for s in  yolo_string.strip().split() ]
    center_x=int(s[1]*size[0])
    center_y=int(s[2]*size[1])
    w=int(s[3]*size[0])
    h=int(s[4]*size[1])
    x1=int(center_x-w/2)
    x2=int(center_x+w/2)
    y1=int(center_y-h/2)
    y2=int(center_y+h/2)
    return (x1,y1),(x2,y2),float(s[5])

In [22]:
%%time
#Примерное время выполнение -> 1 минута
image_path="./data/EmptyPart"
labels_path= "./predict/exp/labels" #Путь к папке с предсказаниями 
submit_name="best_submit"
submit=submit_name+ ".csv"
f=open(submit,"w")
f.write("id,xmin,ymin,xmax,ymax,score\n")
labels_list=os.listdir(labels_path)
labels_list=sorted([int(c[:c.find(".")]) for c in labels_list])
for n in labels_list:
    try:           
      id = str(n)
      l=str(n)+".txt"
      img=cv2.imread(os.path.join(image_path,id+".jpg")) 
      size=(img.shape[1],img.shape[0])    
      file_path=os.path.join(labels_path,l)
      label=open(os.path.join(labels_path,l))
      predicts=[str.strip(s) for s in label.readlines()]
    except:
      pass
    for p in predicts:
        list_to_write=[]
        p1,p2,score=convert_yolo_to_pixels(size,p)
        list_to_write.append(id)
        list_to_write.append(str(p1[0]))
        list_to_write.append(str(p1[1]))
        list_to_write.append(str(p2[0]))
        list_to_write.append(str(p2[1]))
        list_to_write.append(str(score))
        f.write(",".join(list_to_write)+"\n")
    label.close()
f.close()

CPU times: user 45.8 s, sys: 631 ms, total: 46.4 s
Wall time: 46.2 s
